<a href="https://colab.research.google.com/github/compartia/nlp_tools/blob/tokenization-improve/JSON_read.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Инит всега
- юзер, запусти эту ячейку только один раз

In [0]:
#@title  { form-width: "300px", display-mode: "form" }

import json
import subprocess
import sys
import time

import numpy as np
from IPython.core.display import display, HTML
from google.colab import files

Ничто = None
_git_branch = 'tokenization-improve'


def exec(x):
  r = subprocess.check_output(x, shell=True)
  r = r.decode('unicode-escape').encode('latin1').decode('utf8')
  print(r)


print(f"fetching code from GitHub.....{_git_branch}")
try:
  exec('rm -r nlp_tools')
except:
  pass
exec(f'git clone --single-branch --branch {_git_branch} https://github.com/compartia/nlp_tools.git nlp_tools')

print('🦊 GIT revision:')
exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

sys.path.insert(0, 'nlp_tools')

print('❤️importing Code from GitHub ... DONE')



def interactive_upload():
  print(f'Please select .json file:')
  uploaded = files.upload()

  fn = list(uploaded.keys())[0]
  with open(fn, "wb") as _back:
    _back.write(uploaded[fn])
    _back.close()

  with open(fn) as json_file:
    data = json.load(json_file)

  return data



#----
import matplotlib as mpl
from documents import TextMap
from renderer import HtmlRenderer


class ContractDocumentJson:

  def from_json(jsondata):
    c = ContractDocumentJson()
    c.__dict__ = jsondata
    return c

  def __init__(self):
    self.ID = Ничто
    self.filename = Ничто
    self.original_text = Ничто
    self.normal_text = Ничто

    self.import_timestamp = time.time()
    self.analyze_timestamp = time.time()
    self.tokenization_maps = {}
    self.tags = []

 

class DemoRenderer(HtmlRenderer):

  def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
    html = self.to_color_text(tokens, weights, colormap, print_debug, _range)
    display(HTML(html))

  def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None):
    return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range)

   
renderer_ = DemoRenderer()


def print_json_summary(cd):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['$words'])
  print(f'read file {cd.filename}')

  for tag in cd.tags:
    span = tag['span']
    _map = cd.tokenization_maps[tag['span_map']]
    print(tag)
 

 
 
def json2html(cd):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['$words'])
  markup_vector = np.zeros(len(wordsmap))
 
  for tag in cd.tags:
    span = tag['span']
    markup_vector[span[0]:span[1]] += 1

  return renderer_.to_color_text(wordsmap.tokens, markup_vector, _range=(0, 1))
  

# 1. Upload Contract JSON
- пре-запускать для каждого файла

In [0]:
from google.colab import files
data = interactive_upload()

# 2. Render tags

In [0]:
json_obj = ContractDocumentJson.from_json(data)
print_json_summary(json_obj)

In [0]:
html = json2html(json_obj)
display(HTML(html))